In [8]:
import numpy as np
import copy

input = np.array([[0,0],[0,1],[1,0],[1,1]])
output = np.array([[0],[1],[1],[0]])

In [9]:
def rbf(x , c, s):
    #x - datapoint, c - cluster center, s - standard deviation
    return np.exp(-1 / 2 * (np.sqrt((x[0] - c[0])**2 + (x[1] - c[1])**2) / s) ** 2)

In [10]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def der(x):
    return x * (1 - x)

In [11]:
def check_equal(a,b):
    for i in range(len(a)):
        for j in range(len(a[0])):
            if a[i][j] != b[i][j]:
                return False
    return True

In [12]:
def kmeans(X, k):
    #X - datpoints, k - no of clusters
    
    points = X.copy()
    centers = X[0:k].copy()
    previous_centers = X[0:k].copy()

    points_belong_to = [-1 for i in range(len(points))]
    converged = False

    iterations = 0

    while converged != True:
        for i,point in enumerate(points):
            cluster = -1
            min_distance = 1e9

            for curr_cluster,center in enumerate(centers):
                curr_distance = np.sqrt((point[0] - center[0])**2 + (point[1] - center[1])**2)
                if curr_distance < min_distance:
                    min_distance = curr_distance
                    cluster = curr_cluster

            points_belong_to[i] = cluster

        for i in range(k):
            cx,cy = 0,0
            count = 0

            for j in range(len(points)):
                if points_belong_to[j] == i:
                    cx += points[j][0]
                    cy += points[j][1]
                    count += 1
                            
            centers[i][0] = cx/count
            centers[i][1] = cy/count

        if check_equal(previous_centers,centers):
            converged = True

        previous_centers = centers
        iterations += 1

    #print(iterations)

    #find standard deviation
    std = [0 for i in range(k)]

    for i in range(k):
        count = 0
        curr_std = 0

        for j in range(len(points)):
            if points_belong_to[j] == i:
                dist = ((points[j][0] - centers[i][0])**2 + (points[j][1] - centers[i][1])**2)
                curr_std += dist
                count += 1

        curr_std /= count
        std[i] = np.sqrt(curr_std)

    return centers,std

In [13]:
class RBF(object):
    def __init__(self, k=3, lr=0.01, epochs=10000, rbf=rbf, fixed=True):
        self.k = k
        self.lr = lr
        self.epochs = epochs
        self.rbf = rbf
        self.fixed = fixed
        self.w = 2 * np.random.random(k) - 1
        self.b = 2 * np.random.randn(1) - 1

    def fit(self,X,Y,fixed=False):

        if self.fixed == False:
            self.centers,self.stds = kmeans(X,self.k)
        else:
            self.centers,_ = kmeans(X,self.k)
            dMax = max([np.sqrt((c1[0] - c2[0])**2 + (c1[1] - c2[1])**2) for c1 in self.centers for c2 in self.centers])
            self.stds = np.repeat(dMax / np.sqrt(2*self.k), self.k)    

        #training
        for epoch in range(self.epochs):
            curr = []
            for i in range(X.shape[0]):
                # forward pass
                a = np.array([rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
                F = sigmoid(a.T.dot(self.w) + self.b)
                loss = (Y[i] - F).flatten() ** 2
                #print('Loss: {0:.2f}'.format(loss[0]))
                
                # backward pass
                error = -(Y[i] - F).flatten() * der(F)

                # update
                self.w = self.w - self.lr * a * error
                self.b = self.b - self.lr * error
        
    def predict(self, X):
        y_pred = []
        for i in range(X.shape[0]):
            a = np.array([rbf(X[i], c, s) for c, s, in zip(self.centers, self.stds)])
            F = sigmoid(a.T.dot(self.w) + self.b)
            y_pred.append(F)
        return np.array(y_pred)

In [14]:
#driver
rbfn = RBF()
rbfn.fit(input,output)

y_predicted = rbfn.predict(input)
print(y_predicted)

[[0.11612418]
 [0.77877404]
 [0.77924526]
 [0.35010601]]
